In [52]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from neiro_network import pre_process_image, model_classify
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [53]:
def augment_brightness_camera_images(image):
    image1 = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    image1 = np.array(image1, dtype = np.float64)
    random_bright = 0.4+np.random.uniform()
    image1[:,:,2] = image1[:,:,2]*random_bright
    image1[:,:,2][image1[:,:,2]>255]  = 255
    image1 = np.array(image1, dtype = np.uint8)
    image1 = cv2.cvtColor(image1,cv2.COLOR_HSV2RGB)
    return image1

def transform_image(image,ang_range,shear_range,trans_range):
    # Rotation
    ang_rot = np.random.uniform(ang_range)-ang_range/2
    rows,cols,ch = image.shape    
    Rot_M = cv2.getRotationMatrix2D((cols/2,rows/2),ang_rot,1)

    # Translation
    tr_x = trans_range*np.random.uniform()-trans_range/2
    tr_y = trans_range*np.random.uniform()-trans_range/2
    Trans_M = np.float32([[1,0,tr_x],[0,1,tr_y]])

    # Shear
    pts1 = np.float32([[8,8],[40,8],[8,40]])

    pt1 = 8+shear_range*np.random.uniform()-shear_range/2
    pt2 = 40+shear_range*np.random.uniform()-shear_range/2

    pts2 = np.float32([[pt1,8],[pt2,pt1],[8,pt2]])

    shear_M = cv2.getAffineTransform(pts1,pts2)
        
    image = cv2.warpAffine(image,Rot_M,(cols,rows))
    image = cv2.warpAffine(image,Trans_M,(cols,rows))
    image = cv2.warpAffine(image,shear_M,(cols,rows))
    
    image = augment_brightness_camera_images(image)
    image = pre_process_image(image)
    
    return image

In [54]:
path = '../TrainIJCNN2013/'
X_train = np.empty((656, 48, 48, 3), dtype = np.uint8)
y_train = []
X_test = np.empty((203, 48, 48, 3), dtype = np.uint8)
y_test = []

N_classes = 43
i = 0
j = 0

for dir_img in os.listdir(path):
    path_to_dir = path+dir_img
    if os.path.isdir(path_to_dir):
        list_imgs = os.listdir(path_to_dir)
        n_el = len(list_imgs)
        border = n_el//4

        for img in list_imgs[:max(border, 1)]:
            X_test[j]=cv2.resize(cv2.imread(os.path.join(path_to_dir, img)), (48,48))
            y_test.append(int(dir_img))
            j+=1

        for img in list_imgs[border:]:
            X_train[i]=cv2.resize(cv2.imread(os.path.join(path_to_dir, img)), (48,48))
            y_train.append(int(dir_img))
            i+=1

In [55]:
y_train = np_utils.to_categorical(y_train, N_classes) # One-hot encode the labels
y_test = np_utils.to_categorical(y_test, N_classes) # One-hot encode the labels

X_valid = np.array([transform_image(img, 50, 20, 20) for img in X_test])
y_valid = y_test

In [56]:
def generate_arrays_from_file(X_train, y_train):
    ang_range = 0
    shear_range = 0
    trans_range = 0
    y_size = y_train.shape[0]
    while 1:
        ang_range = min(ang_range + 5, 50)
        shear_range = min(shear_range + 2, 20)
        trans_range = min(trans_range + 2, 20)
        
        for i in range(200): 
            indx_batch = np.random.randint(0, y_size, 128)
            X_batch = np.array([transform_image(X_train[indx], ang_range, shear_range, trans_range) for indx in indx_batch])
            yield X_batch, y_train[indx_batch]

In [57]:
model = model_classify()
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
               metrics=['accuracy']) 

checkpointer = ModelCheckpoint(filepath="weights_sign.hdf5", verbose=1, save_best_only=True, save_weights_only=True)
model.fit_generator( generate_arrays_from_file(X_train, y_train),
          steps_per_epoch = 100, epochs = 40,
          verbose=1, validation_data=(X_valid, y_valid), callbacks=[checkpointer])

Epoch 1/40
100/100 [==============================] - 2410s - loss: 1.3893 - acc: 0.6033 - val_loss: 5.3000 - val_acc: 0.1576
Epoch 2/40
100/100 [==============================] - 1297s - loss: 0.1354 - acc: 0.9570 - val_loss: 6.6878 - val_acc: 0.1576
Epoch 3/40
100/100 [==============================] - 1273s - loss: 0.1975 - acc: 0.9396 - val_loss: 5.8965 - val_acc: 0.1921
Epoch 4/40
100/100 [==============================] - 1257s - loss: 0.1022 - acc: 0.9683 - val_loss: 6.3440 - val_acc: 0.1921
Epoch 5/40
100/100 [==============================] - 1304s - loss: 0.2589 - acc: 0.9237 - val_loss: 4.5654 - val_acc: 0.2709
Epoch 6/40
100/100 [==============================] - 1285s - loss: 0.1480 - acc: 0.9548 - val_loss: 4.5555 - val_acc: 0.2562
Epoch 7/40
100/100 [==============================] - 1255s - loss: 0.3004 - acc: 0.9137 - val_loss: 3.4291 - val_acc: 0.3251
Epoch 8/40
100/100 [==============================] - 1295s - loss: 0.2002 - acc: 0.9407 - val_loss: 3.1257 - val_acc:

100/100 [==============================] - 1762s - loss: 0.2798 - acc: 0.9162 - val_loss: 0.3779 - val_acc: 0.8916
Epoch 31/40
100/100 [==============================] - 1607s - loss: 0.2773 - acc: 0.9195 - val_loss: 0.3565 - val_acc: 0.9064
Epoch 32/40
100/100 [==============================] - 1992s - loss: 0.2656 - acc: 0.9177 - val_loss: 0.3567 - val_acc: 0.9212
Epoch 33/40
100/100 [==============================] - 1766s - loss: 0.2599 - acc: 0.9227 - val_loss: 0.3932 - val_acc: 0.8916
Epoch 34/40
100/100 [==============================] - 1993s - loss: 0.2604 - acc: 0.9230 - val_loss: 0.4027 - val_acc: 0.9064
Epoch 35/40
100/100 [==============================] - 1946s - loss: 0.2537 - acc: 0.9248 - val_loss: 0.2919 - val_acc: 0.9064
Epoch 36/40
100/100 [==============================] - 1792s - loss: 0.2371 - acc: 0.9289 - val_loss: 0.3004 - val_acc: 0.9212
Epoch 37/40
100/100 [==============================] - 1739s - loss: 0.2485 - acc: 0.9259 - val_loss: 0.3330 - val_acc: 0.9